<a href="https://colab.research.google.com/github/fernando2393/DT2119-Final-Project/blob/master/DT2119_FinalProject_maria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following code downloads and unzips both dataset and metadata:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pathlib import Path
import librosa.display
import librosa
import sklearn as skl
import os
import warnings
from tqdm import tqdm

In [0]:
"""
This class is in charge of loading the relevant data and splitting the dataset.
note: Check https://nbviewer.jupyter.org/github/mdeff/fma/blob/outputs/usage.ipynb for a deeper understanding of
      the data format in the .csv files.
"""

import pandas as pd
DATASETS = "/content/drive/My Drive/Datasets/fma_reduced/"
METADATA = "/content/drive/My Drive/Datasets/fma_metadata/"
MFCC = "/content/drive/My Drive/Datasets/mfcc.csv"



class Loader:
    def __init__(self):
        self.features = ['mfcc', 'chroma_cens', 'tonnetz', 'spectral_contrast',
                         ['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff'],
                         ['rmse', 'zcr']]  # Main categories of the stored features

    @staticmethod
    def load_features():
        """
        This method loads the data features from the .csv file.
        :return features of the data.
        """
        features = pd.read_csv(METADATA + "features.csv", index_col=0, header=[0, 1, 2])

        return features

    @staticmethod
    def load_echonest():
        """
        This method loads the data echonest features from the .csv file.
        :return echonest features of the data.
        """
        echonest = pd.read_csv(METADATA + "echonest.csv", index_col=0, header=[0, 1, 2])

        return echonest

    @staticmethod
    def load_genres():
        """
        This method loads the data genres from the .csv file.
        :return genres of the data and top level genres.
        """
        genres = pd.read_csv(METADATA + "genres.csv", index_col=0)
        top_level = genres['top_level'].unique()  # This corresponds to the considered "top-level genres"

        print("There is a total of " + str(genres.shape[0]) + " genres.")
        print("There is a total of " + str(len(top_level)) + " top-level genres.")

        return genres, top_level

    @staticmethod
    def load_tracks():
        """
        This method loads the data tracks from the .csv file.
        :return music tracks.
        """
        tracks = pd.read_csv(METADATA + "tracks.csv", index_col=0, header=[0, 1])

        return tracks

    @staticmethod
    def load_mfcc():
        if not Path(MFCC).is_file():  # Check if the file exists
            compute_mfcc = MfccComputation()
            compute_mfcc.preprocessing()  # If the file does not exist, create it
        mfcc_val = pd.read_csv(MFCC)

        return mfcc_val

    @staticmethod
    def split_dataset(tracks, features, cat='mfcc'):
        """
        This methods separates the tracks into dataset by means of the 'cat' feature.
        :param tracks: the music tracks loaded in the format returned by 'load_tracks'.
        :param features: the music features loaded in the format returned by 'load_features'.
        :param cat: the feature category used to obtain the dataset features. It will be 'mfcc' by default.
        :return training set, validation set and test set.
        """
        train = tracks['set', 'split'] == 'training'  # Training songs
        val = tracks['set', 'split'] == 'validation'  # Validation songs
        test = tracks['set', 'split'] == 'test'  # Test songs

        x_train = features.loc[train, cat]
        x_val = features.loc[val, cat]
        x_test = features.loc[test, cat]
        y_train = tracks.loc[train, ('track', 'genre_top')]
        y_val = tracks.loc[val, ('track', 'genre_top')]
        y_test = tracks.loc[test, ('track', 'genre_top')]

        return (x_train, y_train), (x_val, y_val), (x_test, y_test)


In [0]:
def compute_mfcc(filepath):
    """
    This method loads the a music track and computes the mfcc.
    :param filepath: music track.
    :return mfcc of the data track.
    """
    x, sr = librosa.load(filepath, sr=None, mono=True)
    stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
    mel = librosa.feature.melspectrogram(sr=sr, S=stft ** 2)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
    mfcc = skl.preprocessing.StandardScaler().fit_transform(mfcc)

    return mfcc


class MfccComputation:

    @staticmethod
    def preprocessing():
        """
        This method parses the data files, calls compute_mfcc and save them into a .csv file.
        """
        warnings.filterwarnings('ignore')
        folders = os.listdir(DATASETS)
        if '.DS_Store' in folders:  # MacOS file system check
            folders.remove('.DS_Store')
        folders.sort()
        mfcc_dict = {}
        for foldername in tqdm(folders):
            files = os.listdir(DATASETS + foldername)
            files.sort()
            for file in files:
                key = file.strip('0')
                key = key.replace('.mp3', '')
                mfcc = compute_mfcc(DATASETS + foldername + '/' + file)
                mfcc_dict[key] = [mfcc]

        df = pd.DataFrame(list(mfcc_dict.items()), columns=['track', 'mfcc'])
        df.to_csv(MFCC)


In [6]:
loader = Loader()
mfcc = loader.load_mfcc()
#tracks = loader.load_tracks()
#features = loader.load_features()
#(x_train, y_train), (x_val, y_val), (x_test, y_test) = loader.split_dataset(tracks, features)

 77%|███████▋  | 99/129 [1:19:07<34:01, 68.07s/it]

NoBackendError: ignored

# Music Genre Classifier:
Music classifier by genre using convolutional neural networks.

In [0]:
class Classifier:
    @staticmethod
    def build(inputShape):
        
        model = Sequential()

        model.add(Conv1D(32, (3, 3), strides=(1, 1), input_shape=inputShape)) 
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Conv1D(32, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Conv1D(64, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(0.2))
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(classes, activation='softmax'))
        
        print("summary")
        print(model.summary())
        
        return model

In [0]:
inputShape = x_train.shape
inputShape

In [0]:
model = Classifier.build(inputShape)

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# Callbacks: early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1, 
                               patience=10,
                               mode='max',
                               restore_best_weights=True)

filepath = "weights.{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [early_stopping, checkpoint]